## Old notebook

This one was meant to use the model to predict ratings, but we are going to aggregate the human ratings instead

In [1]:
%cd /home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/

import numpy as np;
from embeddings.h5_processor import H5DataProcessor, DatasetConfig, ProcessedDataset, AudioSegmentSplitter;
from sklearn.preprocessing import StandardScaler;
from sklearn.model_selection import train_test_split
import pandas as pd;
from universal_music.models import RatingsClassifier;

/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen/.venv/lib64/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


/home/DAVIDSON/dutuller/Workspace/DRI1/MusicGen


In [2]:
# Process datasets using H5DataProcessor
processor = H5DataProcessor()

config = {
    'dataset': 'embeddings/NHS',
    'subfolder': 'samples/wav'
}

dataset = processor.process_h5_file(
    processor.get_embedding_path(DatasetConfig(**config)),
    DatasetConfig(**config)
)


In [3]:
embeddings = dataset.embeddings
filenames = dataset.filenames

df = pd.read_csv('universal_music/FFfull.csv', low_memory=False)

web_df = df[df['study'] == 'web'].copy()
field_df = df[df['study'] == 'field'].copy()

song_functions = df.groupby("song").agg({'songfunction': 'first'})

# Get ratings and normalize
web_ratings = web_df[["song","danc","heal","baby","love"]]

In [4]:
import numpy as np
import pandas as pd
import re
from sklearn.model_selection import train_test_split

# Function to extract song ID from filename
def extract_song_id(filename):
    # Extract the base filename
    base_name = filename.split('/')[-1]
    # Extract numbers from filename
    match = re.search(r'(\d+)', base_name)
    if match:
        return int(match.group(1))
    return None

# Extract song IDs from filenames
song_ids = [extract_song_id(filename) for filename in filenames]
song_id_to_idx = {song_id: idx for idx, song_id in enumerate(song_ids) if song_id is not None}

# Get all ratings (each row is a rating by one participant for one song)
ratings_df = web_df[["song", "danc", "heal", "baby", "love"]]

# Group by song to count ratings per song
ratings_per_song = ratings_df.groupby('song').size()
print(f"Average ratings per song: {ratings_per_song.mean():.2f}")
print(f"Min ratings per song: {ratings_per_song.min()}")
print(f"Max ratings per song: {ratings_per_song.max()}")

# Get unique song IDs that have embeddings
unique_song_ids = list(set(song_id_to_idx.keys()).intersection(set(ratings_df['song'])))

# Split song IDs into train and test sets
train_song_ids, test_song_ids = train_test_split(unique_song_ids, test_size=0.2, random_state=42)

print(f"Training songs: {len(train_song_ids)}")
print(f"Test songs: {len(test_song_ids)}")

# Create X and y matrices for training and testing
X_train_list, y_train_list = [], []
X_test_list, y_test_list = [], []

# For each rating entry
for _, row in ratings_df.iterrows():
    song_id = row['song']
    
    # Check if we have an embedding for this song
    if song_id in song_id_to_idx:
        embedding_idx = song_id_to_idx[song_id]
        embedding = embeddings[embedding_idx]
        
        # Create rating array
        rating = [
            row['danc'],
            row['heal'],
            row['baby'],
            row['love']
        ]
        
        # Add to appropriate set based on song ID
        if song_id in train_song_ids:
            X_train_list.append(embedding)
            y_train_list.append(rating)
        elif song_id in test_song_ids:
            X_test_list.append(embedding)
            y_test_list.append(rating)

# Convert lists to arrays
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)
X_test = np.array(X_test_list)
y_test = np.array(y_test_list)

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

Average ratings per song: 1094.82
Min ratings per song: 919
Max ratings per song: 1185
Training songs: 94
Test songs: 24
X_train shape: (102698, 1536)
y_train shape: (102698, 4)
X_test shape: (26491, 1536)
y_test shape: (26491, 4)


In [5]:
# Create and configure model
model = RatingsClassifier(max_iter=100)

Using device: cuda


In [6]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [7]:
# Training
model.fit(X_train_scaled, y_train, verbose=True)

Loss: 5.5613: 100%|██████████| 100/100 [01:15<00:00,  1.32it/s]


In [8]:
# Predicting ratings on the test set
pred_ratings = model.predict_ratings(X_test_scaled)